In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score,accuracy_score
import pandas as pd
import numpy as np
from bpe import Encoder

alpha_vals = np.linspace(0.001,1,100)

def tuning_nb(xtrain, ytrain, xdev, ydev):
    alpha_params = {}
    for alpha in alpha_vals:
        model = MultinomialNB(alpha = alpha)
        model.fit(xtrain,ytrain)
        pred = model.predict(xdev)
        alpha_params[alpha] = f1_score(ydev,pred)
    maxAlpha = max(alpha_params, key=alpha_params.get) 
    print('After tuning NB for', len(alpha_vals), 'we get param:', maxAlpha)
    return maxAlpha

def train_and_test_nb(xtrain, ytrain, xtest, ytest, xdev, ydev):
    encoder = Encoder(5000, pct_bpe=0.88)  # params chosen for demonstration purposes
    encoder.fit(xtrain)
    xtrain = [' '.join(encoder.tokenize(name)) for name in xtrain]
    xtest = [' '.join(encoder.tokenize(name)) for name in xtest]
    xdev = [' '.join(encoder.tokenize(name)) for name in xdev]
    
    vectorizer = CountVectorizer(ngram_range=(1, 2), lowercase=False)
    x_train = vectorizer.fit_transform(xtrain)
    x_test = vectorizer.transform(xtest)
    x_dev = vectorizer.transform(xdev)
    
    maxAlpha = tuning_nb(x_train, ytrain, x_dev, ydev)
    clf = MultinomialNB(maxAlpha)
    clf.fit(x_train.toarray(), ytrain)
    pred = clf.predict(x_test.toarray())
    f1score = f1_score(ytest,pred)
    return f1score
    

In [7]:
c_vals = np.linspace(0.001,10,100)

def tuning_lr(xtrain, ytrain, xdev, ydev):
    c_params = {}
    for c in c_vals:
        model = LogisticRegression(C = c)
        model.fit(xtrain,ytrain)
        pred = model.predict(xdev)
        c_params[c] = f1_score(ydev,pred)
    maxC = max(c_params, key=c_params.get) 
    print('After tuning LR for', len(alpha_vals), 'we get param:', maxC)
    return maxC

def train_and_test_lr(xtrain, ytrain, xtest, ytest, xdev, ydev):
    encoder = Encoder(1000, pct_bpe=0.88)  # params chosen for demonstration purposes
    encoder.fit(xtrain)
    xtrain = [' '.join(encoder.tokenize(name)) for name in xtrain]
    xtest = [' '.join(encoder.tokenize(name)) for name in xtest]
    xdev = [' '.join(encoder.tokenize(name)) for name in xdev]
    
    vectorizer = CountVectorizer(ngram_range=(1, 2), lowercase=False)
    x_train = vectorizer.fit_transform(xtrain)
    x_test = vectorizer.transform(xtest)
    x_dev = vectorizer.transform(xdev)
    
    maxC = tuning_lr(x_train, ytrain, x_dev, ydev)
    clf = LogisticRegression(C=maxC)
    clf.fit(x_train.toarray(), ytrain)
    pred = clf.predict(x_test.toarray())
    f1score = f1_score(ytest,pred)
    return f1score

In [47]:
from sklearn.model_selection import GridSearchCV

def tuning_svm(xtrain, ytrain):
    print("Fitting the classifier to the training set")
    param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['rbf', 'linear']}
    clf = GridSearchCV(SVC(), param_grid)
    clf = clf.fit(xtrain, ytrain)
    print("Best estimator found by grid search:")
    print(clf.best_estimator_)
    return clf

def train_and_test_svm(xtrain, ytrain, xtest, ytest, xdev, ydev):
    encoder = Encoder(1000, pct_bpe=0.88)  # params chosen for demonstration purposes
    encoder.fit(xtrain)
    xtrain = [' '.join(encoder.tokenize(name)) for name in xtrain]
    xtest = [' '.join(encoder.tokenize(name)) for name in xtest]
    xdev = [' '.join(encoder.tokenize(name)) for name in xdev]
    
    xtrain += xdev
    ytrain += ydev
    vectorizer = CountVectorizer(ngram_range=(1, 2), lowercase=False)
    x_train = vectorizer.fit_transform(xtrain)
    x_test = vectorizer.transform(xtest)
    
    clf = tuning_svm(x_train, ytrain)
    pred = clf.predict(x_test.toarray())
    f1score = f1_score(ytest,pred)
    return f1score

In [16]:
print('Naive Bayes')
f1s = 0.0
for idx in range(5):
    train = pd.read_excel('data/folds/train'+str(idx)+'.xlsx')
    test = pd.read_excel('data/folds/test'+str(idx)+'.xlsx')
    dev = pd.read_excel('data/folds/dev'+str(idx)+'.xlsx')
    xtrain, ytrain = list(train['minang']), list(train['sentiment'])
    xtest, ytest = list(test['minang']), list(test['sentiment'])
    xdev, ydev = list(dev['minang']), list(dev['sentiment'])
    f1s += train_and_test_nb(xtrain, ytrain, xtest, ytest, xdev, ydev)
print(f1s/5.0)

Naive Bayes
After tuning NB for 100 we get param: 0.112
After tuning NB for 100 we get param: 0.09181818181818183
After tuning NB for 100 we get param: 0.14227272727272727
After tuning NB for 100 we get param: 0.05145454545454546
After tuning NB for 100 we get param: 0.05145454545454546
0.6849413876772914


In [19]:
print('Logistic Regression')
f1s = 0.0
for idx in range(5):
    train = pd.read_excel('data/folds/train'+str(idx)+'.xlsx')
    test = pd.read_excel('data/folds/test'+str(idx)+'.xlsx')
    dev = pd.read_excel('data/folds/dev'+str(idx)+'.xlsx')
    xtrain, ytrain = list(train['minang']), list(train['sentiment'])
    xtest, ytest = list(test['minang']), list(test['sentiment'])
    xdev, ydev = list(dev['minang']), list(dev['sentiment'])
    f1s += train_and_test_lr(xtrain, ytrain, xtest, ytest, xdev, ydev)
print(f1s/5.0)

Logistic Regression


/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

...

/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

After tuning LR for 100 we get param: 6.465000000000001
0.5795027045007121


/home/ffajri/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [49]:
print('SVM')
f1s = 0.0
for idx in range(5):
    train = pd.read_excel('data/folds/train'+str(idx)+'.xlsx')
    test = pd.read_excel('data/folds/test'+str(idx)+'.xlsx')
    dev = pd.read_excel('data/folds/dev'+str(idx)+'.xlsx')
    xtrain, ytrain = list(train['minang']), list(train['sentiment'])
    xtest, ytest = list(test['minang']), list(test['sentiment'])
    xdev, ydev = list(dev['minang']), list(dev['sentiment'])
    f1s += train_and_test_svm(xtrain, ytrain, xtest, ytest, xdev, ydev)
print(f1s/5.0)

SVM
Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Fitting 